In [1]:
from dotenv import load_dotenv
import os

# .env 불러오기
load_dotenv()

# 기본 모델 설정
# 여기서 설정할 수도 있지만 .env 내에 넣어줘도 됨
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

In [2]:
from crewai import Crew, Agent, Task
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, YoutubeChannelSearchTool


search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
# Youtube 채널에 대한 RAG(검생 증강 생성)를 수행함
# OpenAI Embeddings를 사용함
# Agent가 알아서 어떤 채널을 검색할지 직접 찾음
youtube_tool = YoutubeChannelSearchTool()
# Agent가 어떤 채널을 봐야 하는지 지정해줄 수 있음
# youtube_tool = YoutubeChannelSearchTool(youtube_channel_handle="exampleChannel")

# Researcher, Marketer, Writer Agent 생성

researcher = Agent(
    role="Senior Researcher",
    goal="Search the web, extract and analyze information.",
    backstory="""
    You produce the highest quality research possible.
    You use multiple sources of information and you always double check your sources to make sure they are true and up to date.
    You want to impress your coworkers with your work.
    """,
    allow_delegation=False,
    tools=[
        search_tool,
        scrape_tool,
        youtube_tool,
    ],
    max_iter=10,
    verbose=True,
)

marketer = Agent(
    role="Senior Marketer",
    goal="Come up with ideas that generate viral and useful content.",
    backstory="""
    You work at a marketing agency.
    You are the best at coming up with ideas to make content go viral.
    Your ideas are used for video, advertising, social media marketing, the content you produce appeals to a young audience.
    """,
    allow_delegation=True,
    verbose=True,
)

writer = Agent(
    role="Senior Writer",
    goal="Write scripts for viral Youtube videos.",
    backstory="""
    You write scripts for videos that keep people engaged and entertained.
    Your content is easy and fun to watch, it is informative and it makes people want to share it with their friends.
    You are working for a very important client.
    """,
    allow_delegation=True,
    verbose=True,
)

In [3]:
# brainstorm -> selection -> [research, competitors, inspiration]: 실행 순서 무관 -> script

# 5개의 아이디어를 생성
brainstorm_task = Task(
    description="Come up with 5 video ideas for a Youtube channel in the {industry} industry",
    agent=marketer,
    expected_output="Your answer MUST be a list of 5 ideas for a Youtube video with an explanation of what the angle of the video would be.",
    output_file="ideas_task.md",
    # 다음 설정을 통해 사람에게 다음 단계로 넘어가도 되는지 반드시 허락을 받도록 할 수 있음
    human_input=True,
)

# 5개의 아이디어 중 하나를 선택
selection_task = Task(
    description="Select a video idea that has the highest potential of going viral.",
    agent=writer,
    expected_output="Your answer MUST include the idea that was selected as well as an explanation of why that selection was made.",
    human_input=True,
    # brainstorm이 완료되어야 이 작업을 수행함
    context=[
        brainstorm_task,
    ],
    output_file="selection_task.md",
)

# 선택한 아이디어에 대한 자료 조사
research_task = Task(
    description="Do all the research required to write the script of a medium length video about the selected idea.",
    agent=researcher,
    expected_output="You answer must have all the information a writer would need to write a Youtube script.",
    # async_execution이 True로 설정된 작업을 동시해 수행함
    async_execution=True,
    # selection_task가 완료되어야만 이 작업을 수행함
    context=[
        selection_task,
    ],
    output_file="research_task.md",
)

# 선택한 아이디어에 대한 경쟁사의 유튜브 채널이나 관련 글을 조사
# 경쟁사와 비교해 우리가 더 유니크해질 수 있는 방법을 찾도록 함
competitors_task = Task(
    description="Search for videos or articles in the {industry} industry that are similar to the video idea we are working on and suggest ways our video can be different from theirs.",
    expected_output="Your answer must have a list of suggestions writers can follow to make sure the video is as unique and as different from competitors as possible.",
    agent=researcher,
    async_execution=True,
    context=[
        selection_task,
    ],
    output_file="competitors_task.md",
)

# 다른 산업군을 살펴보면서 어떤 방식을 취하고 있고 우리의 비디오에는 어떻게 적용하면 좋을지 생각하도록 함
inspiration_task = Task(
    description="Search for videos or articles that are similar to the video idea we are working on but from other industries.",
    expected_output="Your answer must have a list of examples of articles and videos that have a similar angle as the video we are making but that are in different industries.",
    agent=researcher,
    async_execution=True,
    context=[
        selection_task,
    ],
    output_file="inspiration_task.md",
)

# selection, research, competitors, inspiration이 모두 완료되면 스크립트를 작성하게 함
script_task = Task(
    description="Write the script for a Youtube video for a channel in the {industry} industry.",
    expected_output="A script for a Youtube video with a title, an introduction, at least three sections, and an outro. Make sure to also include the prompt to generate a thumbnail for the video.",
    agent=writer,
    # context에 넣어준 모든 작업이 마쳐야지만 이 작업을 수행함
    # 즉, 여기에 넣어준 작업의 결과물이 있어야 이 작업을 수행할 수 있다는 것을 명시하는 것
    context=[
        selection_task,
        research_task,
        competitors_task,
        inspiration_task,
    ],
    output_file="script_task.md",
)

crew = Crew(
    agents=[
        researcher,
        writer,
        marketer,
    ],
    tasks=[
        brainstorm_task,
        selection_task,
        research_task,
        inspiration_task,
        competitors_task,
        script_task,
    ],
    verbose=True,
)

result = crew.kickoff(
    inputs=dict(
        industry="Hot Sauce",
    ),
)

result

# Agent: Senior Marketer
## Task: Come up with 5 video ideas for a Youtube channel in the Hot Sauce industry
 

I encountered an error while trying to use the tool. This was the error: Arguments validation failed: 2 validation errors for DelegateWorkToolSchema
task
  Input should be a valid string [type=string_type, input_value={'description': 'Come up ...dience.', 'type': 'str'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type
context
  Input should be a valid string [type=string_type, input_value={'description': 'The goal...lenges.', 'type': 'str'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type.
 Tool Delegate work to coworker accepts these inputs: Tool Name: Delegate work to coworker
Tool Arguments: {'task': {'description': 'The task to delegate', 'type': 'str'}, 'context': {'description': 'The context for the task', 'type': 'str'}, 'coworker': {'description': 'The role/name of the 

CrewOutput(raw='**Script for "Hot Sauce Roulette Challenge"**\n\n**[INTRO - 0:00 - 0:30]**\n- **Camera focuses on host standing next to a colorful wheel filled with various hot sauces.**\n- Host: "Welcome back to our channel! Today, we’re about to ignite the taste buds with the ‘Hot Sauce Roulette Challenge!’ Participants will spin this wheel and face the ultimate hot sauce showdown— will they thrive or dive into the fiery abyss?"\n\n**[SETTING UP THE CHALLENGE - 0:31 - 1:00]**\n- **Close-up of the wheel with sauces labeled from Mild to Scorching Inferno.**\n- Host: "Here’s how this works: each participant spins the wheel and must try whatever sauce it lands on. From mild and creamy to blazing hot, the suspense will keep you on the edge of your seat!"\n\n**[PARTICIPANTS INTRODUCTION - 1:01 - 1:30]**\n- **Cuts to participants prepping themselves, giggling nervously.**\n- Host: “Joining us today are some brave friends and popular influencers! Let\'s meet [insert participant names]! Who w